In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import re
import nltk
from nltk import word_tokenize, pos_tag

In [2]:
df=spark.read.parquet("gs://bbm-labs-ds-samples/content_classification/dmoz/train.parquet").rdd.repartition(32).toDF()

In [3]:
#statistics
df = df.withColumn("content", regexp_replace(col("content"), "(?<=(?<!\pL)\pL) (?=\pL(?!\pL))", ""))
df = df.withColumn("content",regexp_replace(col("content"), "(\s)+"," "))
df = df.withColumn("size",length(col("content")))
split_col = pyspark.sql.functions.split(df['content'], ' ')
df = df.withColumn("size",size(split_col))

In [4]:
data_max = df.groupBy("category").agg(max("size").alias("max")).orderBy("category")
data_min = df.groupBy("category").agg(min("size").alias("min")).orderBy("category")

In [5]:
df = df.where(col('category')=='Hobbies').where(col("content").startswith("Donkey Kong Country Returns")).where(col("size") ==1882)

In [234]:
# df.write.parquet('/root/notebooks/testing01.parquet')
data=spark.read.parquet("/root/notebooks/testing01.parquet")


In [245]:
#Join word with single spaces
data = data.withColumn("content", regexp_replace(col("content"), "(?<=(?<!\pL)\pL) (?=\pL(?!\pL))", ""))
#replace more than one special character with single character of the same 
#(hyphen-)
data = data.withColumn("content",regexp_replace(col("content"), "\-(?=\-)|\G(?!^)\-"," "))
#(dots.)
data = data.withColumn("content",regexp_replace(col("content"), "\.(?=\.)|\G(?!^)\."," "))
#,,,
data = data.withColumn("content",regexp_replace(col("content"), "\,(?=\,)|\G(?!^)\,"," "))
data = data.withColumn("content",regexp_replace(col("content"), "\,"," ,"))
data = data.withColumn("content",regexp_replace(col("content"), "\."," ."))
#replace (s) with spaces
data = data.withColumn("content",regexp_replace(col("content"),"\(.[a-zA-Z]{0,3}\)"," "))

#deal with markup texts
data = data.withColumn("content",regexp_replace(col("content"),"(\\\\.[*]{0})"," "))
#remove emails from text
data = data.withColumn("email_address",regexp_extract(col('content'), '[A-Za-z]*\.com',0))
data = data.withColumn("content",regexp_replace(col('content'), '[A-Za-z]*\.com',""))
#replace mupltiple spaces from text
data = data.withColumn("content",regexp_replace(col("content"), "(\s)+"," "))

In [239]:
#dealing with contractions
dict_contract ={}
contractions = ['I\'m','you\'re','he\'s','she\'s','we\'re','they\'re','isn\'t','aren\'t','can\'t','don\'t','who\'s','won\'t','they\'d']
abbrevations = ['I am', 'you are','he is','she is','we are','they are','is not','are not','can not','do not','who is','will not','they would']


for i in range(0,len(contractions)):
    dict_contract[contractions[i]] = abbrevations[i]
                

def apostophes(raw):
    res = ''
    for text in raw.split():
        for key in dict_contract:
            if text.lower() == key:
                text = dict_contract[text]
        res += text + " "
                
    return(res)
apos_words = udf(apostophes,StringType())
data = data.withColumn("content",apos_words("content"))
data = data.withColumn("content",regexp_replace(col("content"), "(\s)+"," "))

In [240]:
data.select('content').show(1,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [229]:
#Identifying parts of speech
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
#from nltk import pos_tag
from nltk.tag.perceptron import PerceptronTagger
import string
import re
tagger = PerceptronTagger()
from nltk import pos_tag, word_tokenize
def tag_and_remove(data_str):
    import nltk
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags

    # break string into 'words'
    
    text = data_str.split()

    # tag the text and keep only those with the right tags
    tagged_text = nltk.pos_tag(text)
    return tagged_text

    
tag_and_remove_udf = udf(tag_and_remove,ArrayType(ArrayType(StringType())))
data = data.withColumn("tagged_text", tag_and_remove_udf("content"))

In [199]:
data.select('tagged_text').show(1,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [291]:
#combination of characters and letters
#starts with a special character followed by word
# # data = data.withColumn('content',regexp_replace(col("content"), "\%",""))
# data = data.withColumn('content',regexp_replace(col("content"), "\'t"," not"))

#dealing with adjoined words
from nltk.tag import StanfordNERTagger
st = StanfordNERTagger("/usr/share/stanford-ner/stanford-ner-2014-06-16/classifiers/english.all.3class.distsim.crf.ser.gz","/usr/share/stanford-ner/stanford-ner-2014-06-16/stanford-ner.jar")
dict_words = open('/root/notebooks/new_dictwords.txt',"r").read()
abbreviations =  open('/root/notebooks/abbreviations.txt',"r").read()

def adjoined_words(raw):
    after_adjoined = ''
    wrong_text = ''
    tags = st.tag(raw.split())
    i = 0 
    for tag in tags:
        pos_tag = nltk.pos_tag(tag[0].split())
        for sub_pos in pos_tag:
            if tag[1]=='PERSON': 
                s6 = tag[0]
                flag = 'True'
            elif tag[0].isdigit():
                s6 = tag[0]
                flag = 'True'
            elif tag[0].isupper():
                abbreviations = tag[0]
                if tag[0] in abbreviations.split():
                    s6 = tag[0]
                    flag = 'True'
                else:
                    len_text = len(tag[0])
                    flag = 'False'
                    len_text = len(tag[0])
                    for i in range(1,len_text):
                        if tag[0][0:i].lower() in abbreviations.split():
                            new_text = tag[0][0:i]
                            if new_text.lower() in abbreviations.split():
                                s6 = tag[0:i] + " " + tag[i:]
                                flag = 'True'       
            elif tag[0].lower() in dict_words.lower().split():
                s6 = tag[0]
                flag = 'True'
            elif sub_pos[1] == 'NNP':
                flag = 'False'
                len_text = len(tag[0])
                for i in range(1,len_text):
                    if tag[0][0:i].lower() in dict_words.lower().split():
                            s6 = tag[0][0:i] + " " + tag[0][i:]
                            flag = 'True'
            else:
                s6 = tag[0]
                flag = 'True'           
            if flag == 'True':
                after_adjoined += s6 + ' '
            else:
                s6 = tag[0]
                after_adjoined += s6 + ' '
    return(wrong_text)
res_words = udf(adjoined_words,StringType())
data = data.withColumn("new_content",res_words("content"))

In [292]:
data.select('new_content').show(1,False)

+-----------+
|new_content|
+-----------+
|           |
+-----------+

